# Phi-4 Mini ONNX 平行函數呼叫教學

本筆記本展示如何使用 Phi-4 Mini 搭配 ONNX Runtime GenAI 進行平行函數呼叫。函數呼叫能讓模型根據使用者需求智能地調用外部工具和 API。

## 概述

在本教學中，您將學習如何：
- 設置 Phi-4 Mini 與 ONNX Runtime GenAI
- 定義預訂航班和酒店的函數結構
- 使用 Lark 語法進行引導生成以獲得結構化輸出
- 執行平行函數呼叫以處理複雜的旅行預訂場景

## 先決條件

在執行本筆記本之前，請確保您已：
- 下載 Phi-4 Mini ONNX 模型
- 安裝 `onnxruntime-genai` 套件
- 基本了解函數呼叫的概念


## 第一步：匯入所需的函式庫

首先，我們將匯入實現函式呼叫所需的函式庫。


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## 步驟 2：模型設置與配置

現在我們將配置 Phi-4 Mini ONNX 模型。請確保將路徑替換為您實際的模型目錄。


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## 第三步：設定生成參數

設定生成參數以控制模型的輸出行為。這些設定可確保模型在呼叫函數時提供具決定性且專注的回應。


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## 第四步：定義可用的功能

在這裡，我們定義了 AI 助手可以調用的功能。在此範例中，我們有兩個與旅行相關的功能：
1. **booking_flight_tickets**：用於預訂機場之間的航班
2. **booking_hotels**：用於預訂酒店住宿

這些功能的定義遵循 OpenAI 的功能調用架構格式。


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## 第五步：語法生成輔助函數

這些輔助函數將我們的函數定義轉換為 Lark 語法格式，ONNX Runtime GenAI 使用該格式進行引導生成。這確保模型輸出符合正確 JSON 格式的有效函數調用。


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## 第六步：測試語法生成

讓我們測試語法生成函數，看看它們如何將工具定義轉換為正確的格式。


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## 步驟 7：準備系統提示和生成器

現在我們將建立系統提示，告訴模型可用的工具，並設置生成器的引導生成參數。


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## 第八步：使用引導生成初始化生成器

現在我們將使用已配置的參數創建生成器，並應用 Lark 語法進行引導生成。


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## 第九步：測試平行函數呼叫

現在讓我們使用一個複雜的旅行預訂情境來測試我們的設置，這需要呼叫多個函數。


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### 生成函數調用

模型現在將根據使用者的請求生成結構化的函數調用。透過引導生成，輸出將是有效的 JSON 格式，可直接執行。

**預期輸出**：模型應生成以下函數調用：
1. `booking_flight_tickets`，包含北京 (PEK) 至巴黎 (CDG) 的詳細資訊
2. `booking_hotels`，包含巴黎住宿的詳細資訊

執行下面的單元格以查看即時生成：


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## 結論

🎉 **恭喜！** 你已成功使用 ONNX Runtime GenAI 實現 Phi-4 Mini 的平行函數呼叫。

### 你學到了什麼：

1. **模型設定**：如何使用 ONNX Runtime GenAI 配置 Phi-4 Mini
2. **函數定義**：如何定義 AI 函數呼叫的函數結構
3. **引導生成**：如何使用 Lark 語法生成結構化輸出
4. **平行函數呼叫**：如何處理需要多個函數呼叫的複雜請求

### 主要優勢：

- ✅ **結構化輸出**：引導生成確保有效的 JSON 函數呼叫
- ✅ **平行處理**：在單一請求中處理多個函數呼叫
- ✅ **高效能**：ONNX Runtime 提供最佳化的推理性能
- ✅ **靈活結構**：輕鬆新增或修改函數定義

### 資源：

- [Phi-4 Mini 文件](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [ONNX Runtime GenAI 文件](https://onnxruntime.ai/docs/genai/)
- [函數呼叫最佳實踐](https://platform.openai.com/docs/guides/function-calling)



---

**免責聲明**：  
本文件已使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。雖然我們致力於提供準確的翻譯，但請注意，自動翻譯可能包含錯誤或不準確之處。原始文件的母語版本應被視為權威來源。對於關鍵資訊，建議使用專業人工翻譯。我們對因使用此翻譯而引起的任何誤解或誤釋不承擔責任。
